In [78]:
import random
import tempfile
from pathlib import Path

import pandas as pd
from datasetsforecast.m4 import M4
from utilsforecast.plotting import plot_series

In [79]:
await M4.async_download("data", group="Hourly")
df, *_ = M4.load("data", "Hourly")
uids = df["unique_id"].unique()
random.seed(0)

# sample a subset for demo purposes
sample_uids = random.choices(uids, k=4)
df = df[df["unique_id"].isin(sample_uids)].reset_index(drop=True)
# df['ds'] = df['ds'].astype('int64')

# assume df has columns: unique_id, ds, y
start = pd.Timestamp("2000-01-01 00:00:00")

df["ds"] = start + pd.to_timedelta(df["ds"], unit="h")
df.head()

,unique_id,ds,y
0,H196,2000-01-01 01:00:00,11.8
1,H196,2000-01-01 02:00:00,11.4
2,H196,2000-01-01 03:00:00,11.1
3,H196,2000-01-01 04:00:00,10.8
4,H196,2000-01-01 05:00:00,10.6


In [80]:
df.loc[df["unique_id"] == "H196"].tail()

,unique_id,ds,y
1003,H196,2000-02-11 20:00:00,19.7
1004,H196,2000-02-11 21:00:00,18.6
1005,H196,2000-02-11 22:00:00,17.8
1006,H196,2000-02-11 23:00:00,17.3
1007,H196,2000-02-12 00:00:00,16.8


In [81]:
fig = plot_series(df, max_insample_length=24 * 14, engine="plotly")
fig

In [82]:
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences

In [83]:
fcst = MLForecast(
    models=[],  # we're not interested in modeling yet
    freq="h",
    target_transforms=[Differences([24])],
)
prep = fcst.preprocess(df)
prep.head()

,unique_id,ds,y
24,H196,2000-01-02 01:00:00,0.3
25,H196,2000-01-02 02:00:00,0.3
26,H196,2000-01-02 03:00:00,0.1
27,H196,2000-01-02 04:00:00,0.2
28,H196,2000-01-02 05:00:00,0.2


In [84]:
plot_series(prep, engine="plotly")


# Adding features

## Lags

Looks like the seasonality is gone, we can now try adding some lag features.


In [85]:
fcst = MLForecast(
    models=[],
    freq="h",
    lags=[1, 24],
    target_transforms=[Differences([24])],
)
prep = fcst.preprocess(df)
prep.head()

,unique_id,ds,y,lag1,lag24
48,H196,2000-01-03 01:00:00,0.1,0.1,0.3
49,H196,2000-01-03 02:00:00,0.1,0.1,0.3
50,H196,2000-01-03 03:00:00,0.2,0.1,0.1
51,H196,2000-01-03 04:00:00,0.1,0.2,0.2
52,H196,2000-01-03 05:00:00,0.1,0.1,0.2


In [86]:
prep.drop(columns=["unique_id", "ds"]).corr()["y"]

y        1.000000
lag1     0.622531
lag24   -0.234268
Name: y, dtype: float64

## Lag transforms

Lag transforms are defined as a dictionary where the keys are the lags and the values are the transformations that we want to apply to that lag. The lag transformations can be either objects from the `mlforecast.lag_transforms` module or `numba` jitted functions (so that computing the features doesn’t become a bottleneck and we can bypass the GIL when using multithreading), we have some implemented in the `window-ops package` but you can also implement your own.

In [87]:
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from numba import njit
from window_ops.rolling import rolling_mean

In [88]:
@njit
def rolling_mean_48(x):
    return rolling_mean(x, window_size=48)


fcst = MLForecast(
    models=[],
    freq="h",
    target_transforms=[Differences([24])],
    lag_transforms={
        1: [ExpandingMean()],
        24: [RollingMean(window_size=48), rolling_mean_48],
    },
)
prep = fcst.preprocess(df)
prep.head()

,unique_id,ds,y,expanding_mean_lag1,rolling_mean_lag24_window_size48,rolling_mean_48_lag24
95,H196,2000-01-05 00:00:00,0.1,0.174648,0.150000,0.150000
96,H196,2000-01-05 01:00:00,0.3,0.173611,0.145833,0.145833
97,H196,2000-01-05 02:00:00,0.3,0.175342,0.141667,0.141667
98,H196,2000-01-05 03:00:00,0.3,0.177027,0.141667,0.141667
99,H196,2000-01-05 04:00:00,0.3,0.178667,0.141667,0.141667


## Date features

If your time column is made of timestamps then it might make sense to extract features like week, dayofweek, quarter, etc. You can do that by passing a list of strings with pandas time/date components. You can also pass functions that will take the time column as input, as we’ll show here.

In [89]:
fcst = MLForecast(
    models=[],
    freq="h",
    target_transforms=[Differences([24])],
    date_features=["hour"],
)
fcst.preprocess(df).head()

,unique_id,ds,y,hour
24,H196,2000-01-02 01:00:00,0.3,1
25,H196,2000-01-02 02:00:00,0.3,2
26,H196,2000-01-02 03:00:00,0.1,3
27,H196,2000-01-02 04:00:00,0.2,4
28,H196,2000-01-02 05:00:00,0.2,5


## Target transformations

If you want to do some transformation to your target before computing the features and then re-apply it after predicting you can use the `target_transforms` argument, which takes a list of transformations. You can find the implemented ones in `mlforecast.target_transforms` or you can implement your own as described in the **target transformations guide**.

In [90]:
from mlforecast.target_transforms import LocalStandardScaler

In [91]:
fcst = MLForecast(
    models=[], freq="h", lags=[1], target_transforms=[LocalStandardScaler()]
)
fcst.preprocess(df).head()

,unique_id,ds,y,lag1
1,H196,2000-01-01 02:00:00,-1.493026,-1.383286
2,H196,2000-01-01 03:00:00,-1.575331,-1.493026
3,H196,2000-01-01 04:00:00,-1.657635,-1.575331
4,H196,2000-01-01 05:00:00,-1.712505,-1.657635
5,H196,2000-01-01 06:00:00,-1.794810,-1.712505


In [92]:
from sklearn.base import BaseEstimator


class Naive(BaseEstimator):
    def fit(self, X, y):
        return self

    def predict(self, X):
        return X["lag24"]

In [93]:
fcst = MLForecast(
    models=[Naive()], freq="h", lags=[24], target_transforms=[LocalStandardScaler()]
)
fcst.fit(df)
preds = fcst.predict(1)
preds.head()

,unique_id,ds,Naive
0,H196,2000-02-12 01:00:00,16.2
1,H256,2000-02-12 01:00:00,13.2
2,H381,2000-02-12 01:00:00,244.0
3,H413,2000-02-12 01:00:00,25.0



# Training

Once you’ve decided the features, transformations and models that you want to use you can use the MLForecast.fit method instead, which will do the preprocessing and then train the models. The models can be specified as a list (which will name them by using their class name and an index if there are repeated classes) or as a dictionary where the keys are the names you want to give to the models, i.e. the name of the column that will hold their predictions, and the values are the models themselves.

In [94]:
import lightgbm as lgb

In [95]:
lgb_params = {
    "verbosity": -1,
    "num_leaves": 512,
}

fcst = MLForecast(
    models={
        "LGBMRegressor": lgb.LGBMRegressor(**lgb_params),
        "LGBMRegressor-lo-80": lgb.LGBMRegressor(
            **lgb_params, objective="quantile", alpha=0.10
        ),
        "LGBMRegressor-hi-80": lgb.LGBMRegressor(
            **lgb_params, objective="quantile", alpha=0.90
        ),
    },
    freq="h",
    target_transforms=[Differences([24])],
    lags=[1, 24],
    lag_transforms={
        1: [ExpandingMean()],
        24: [RollingMean(window_size=48)],
    },
    date_features=["hour"],
)
fcst.fit(df)

MLForecast(models=[LGBMRegressor, LGBMRegressor-lo-80, LGBMRegressor-hi-80], freq=h, lag_features=['lag1', 'lag24', 'expanding_mean_lag1', 'rolling_mean_lag24_window_size48'], date_features=['hour'], num_threads=1)

In [96]:
preds = fcst.predict(24)
preds.head()

,unique_id,ds,LGBMRegressor,LGBMRegressor-lo-80,LGBMRegressor-hi-80
0,H196,2000-02-12 01:00:00,16.295257,16.242562,16.400777
1,H196,2000-02-12 02:00:00,15.910282,15.834486,16.084133
2,H196,2000-02-12 03:00:00,15.728367,15.645285,15.894385
3,H196,2000-02-12 04:00:00,15.468414,15.365720,15.589312
4,H196,2000-02-12 05:00:00,15.081279,14.949564,15.186851


In [97]:
fig = plot_series(df, preds, max_insample_length=24 * 7, level=[80], engine="plotly")

# Update the layout to use 'x unified' hovermode
fig.update_layout(hovermode="x unified")

fig.show()

In [98]:
preds.head()

,unique_id,ds,LGBMRegressor,LGBMRegressor-lo-80,LGBMRegressor-hi-80
0,H196,2000-02-12 01:00:00,16.295257,16.242562,16.400777
1,H196,2000-02-12 02:00:00,15.910282,15.834486,16.084133
2,H196,2000-02-12 03:00:00,15.728367,15.645285,15.894385
3,H196,2000-02-12 04:00:00,15.468414,15.365720,15.589312
4,H196,2000-02-12 05:00:00,15.081279,14.949564,15.186851


# Save the model for future re-use

In [99]:
with tempfile.TemporaryDirectory() as tmpdir:
    save_dir = Path.cwd() / "mlforecast"
    fcst.save(save_dir)
    fcst2 = MLForecast.load(save_dir)
    preds2 = fcst2.predict(24)
    pd.testing.assert_frame_equal(preds, preds2)

# Estimating model performance

## Cross validation
In order to get an estimate of how well our model will be when predicting future data we can perform cross validation, which consist on training a few models independently on different subsets of the data, using them to predict a validation set and measuring their performance.
Since our data depends on time, we make our splits by removing the last portions of the series and using them as validation sets. This process is implemented in `MLForecast.cross_validation`.

In [100]:
fcst = MLForecast(
    models=lgb.LGBMRegressor(**lgb_params),
    freq="h",
    target_transforms=[Differences([24])],
    lags=[1, 24],
    lag_transforms={
        1: [ExpandingMean()],
        24: [RollingMean(window_size=48)],
    },
    date_features=["hour"],
)
cv_result = fcst.cross_validation(
    df,
    n_windows=4,  # number of models to train/splits to perform
    h=24,  # length of the validation set in each window
)
cv_result

,unique_id,ds,cutoff,y,LGBMRegressor
0,H196,2000-02-08 01:00:00,2000-02-08,15.8,15.914186
1,H196,2000-02-08 02:00:00,2000-02-08,15.4,15.398749
2,H196,2000-02-08 03:00:00,2000-02-08,15.1,15.075451
3,H196,2000-02-08 04:00:00,2000-02-08,14.8,14.748062
4,H196,2000-02-08 05:00:00,2000-02-08,14.5,14.405898
...,...,...,...,...,...
379,H413,2000-02-11 20:00:00,2000-02-11,99.0,66.580431
380,H413,2000-02-11 21:00:00,2000-02-11,88.0,37.224494
381,H413,2000-02-11 22:00:00,2000-02-11,47.0,33.639789
382,H413,2000-02-11 23:00:00,2000-02-11,41.0,36.016230


In [101]:
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mae, rmse, smape

In [102]:
cv_result.head()

,unique_id,ds,cutoff,y,LGBMRegressor
0,H196,2000-02-08 01:00:00,2000-02-08,15.8,15.914186
1,H196,2000-02-08 02:00:00,2000-02-08,15.4,15.398749
2,H196,2000-02-08 03:00:00,2000-02-08,15.1,15.075451
3,H196,2000-02-08 04:00:00,2000-02-08,14.8,14.748062
4,H196,2000-02-08 05:00:00,2000-02-08,14.5,14.405898


### Define the metrics you want to see

In [103]:
from typing import Callable, List

import pandas as pd
from utilsforecast.evaluation import evaluate


def define_metrics(df: pd.DataFrame, metrics: List[Callable]) -> pd.DataFrame:
    """
    Returns a DataFrame with the global metrics for the entire dataset.

    Parameters:
    df (pd.DataFrame): Cross-validation results (must contain 'y' and model columns).
    metrics (List[Callable]): List of metric functions (e.g., [mae, rmse]).
    """
    # copy df to keep the original intect
    df_global = df.copy()

    # We have to drop the cut-off column to avoid
    if "cutoff" in df_global.columns:
        df_global = df_global.drop(columns=["cutoff"])

    # Force Global Aggregation
    # We set unique_id to 'Total' so evaluate() treats all rows as a single group
    df_global["unique_id"] = "Total"

    # 4. Calculate Metrics
    results = evaluate(df_global, metrics=metrics)

    # drop the unique_id column
    if "unique_id" in results.columns:
        results = results.drop(columns=["unique_id"])

    return results


# Assuming 'cv_result' is your cross-validation dataframe
metrics_df = define_metrics(cv_result, metrics=[mae, rmse, smape])

metrics_df

,metric,LGBMRegressor
0,mae,12.361266
1,rmse,30.767151
2,smape,0.069878


### Try improving the results

In [104]:
from mlforecast.lag_transforms import ExponentiallyWeightedMean

In [105]:
fcst = MLForecast(
    models=lgb.LGBMRegressor(**lgb_params),
    freq="h",
    lags=[1, 24],
    lag_transforms={
        1: [ExponentiallyWeightedMean(alpha=0.5)],
        24: [RollingMean(window_size=48)],
        48: [ExponentiallyWeightedMean(alpha=0.9)],
    },
    date_features=["hour"],
)
cv_result2 = fcst.cross_validation(
    df,
    n_windows=4,
    h=48,
)

# Evaluate
metrics_df = define_metrics(cv_result2, metrics=[mae, rmse, smape])

metrics_df

,metric,LGBMRegressor
0,mae,11.957942
1,rmse,25.869702
2,smape,0.082583


### Visualise per cut-off date


In [106]:
cv_result.head()

,unique_id,ds,cutoff,y,LGBMRegressor
0,H196,2000-02-08 01:00:00,2000-02-08,15.8,15.914186
1,H196,2000-02-08 02:00:00,2000-02-08,15.4,15.398749
2,H196,2000-02-08 03:00:00,2000-02-08,15.1,15.075451
3,H196,2000-02-08 04:00:00,2000-02-08,14.8,14.748062
4,H196,2000-02-08 05:00:00,2000-02-08,14.5,14.405898


In [107]:
# find distinct dates from the cv df
distinct_cut_off_dates = sorted(cv_result["cutoff"].drop_duplicates().to_list())

# iterate over the dates 
for i, cutoff_date in enumerate(distinct_cut_off_dates):
    df_plot = df.loc[df["ds"] <= cutoff_date]
    preds_cv = cv_result.loc[cv_result["cutoff"] == cutoff_date]

    fig = plot_series(
        df_plot,
        preds_cv.drop('cutoff', axis=1),
        # max_insample_length=24 * 7 + (7 * i) ,  # change this to keep less dates
        # level=[80],
        engine="plotly",
        palette="tab10"
    )

    # Update the layout to use 'x unified' hovermode
    fig.update_layout(
        title=f"Plot for cutoff date: {cutoff_date}",
        title_font_size=20,
        # Increasing it to 100 or 120 usually fixes the crop.
        margin=dict(t=120),
        hovermode="x unified"
    )

    fig.update_xaxes(range=[ max(distinct_cut_off_dates) - pd.Timedelta(days=9), max(distinct_cut_off_dates) + pd.Timedelta(days=1)])
    
    fig.show()